In [ ]:
from lecture import *

# <center> Introduction to programming in Python </center>
### <center> [Nicolas Barral](http://www.imperial.ac.uk/people/n.barral), [Gerard Gorman](http://www.imperial.ac.uk/people/g.gorman) </center>

# <center> Lecture 6: Reading files </center>

## Learning objectives: 
<hr style="border: solid 2px red; margin-top: 1.5% ">

* open and close a file
* read one column data files
* read multi-column data files 
* skip ill-formed lines
* write data to a file

## Reading data from a plain text file
<hr style="border: solid 2px red; margin-top: 1.5% ">

We can read text from a [text file](http://en.wikipedia.org/wiki/Text_file) into strings in a program. This is a common (and simple) way for a program to get input data. The basic recipe is:
```python
# Open text file
infile = open("myfile.dat", "r")

# Read next line:
line = infile.readline()

# Read the lines in a loop one by one:
for line in infile:
    <process line>

# Load all lines into a list of strings:
lines = infile.readlines()
for line in lines:
    <process line>```

Let's look at the file [./data/data1.txt](./data/data1.txt) (all of the data files in this lecture are stored in the sub-folder *data/* of this notebook library). The files has a column of numbers:
```
21.8
18.1
19
23
26
17.8```

The goal is to read this file and calculate the mean:

In [ ]:
# Open data file
infile = open("data/data1.txt", "r")

# Initialise values
mean = 0
n=0

# Loop to perform sum
for number in infile:
    number = float(number)
    mean = mean + number
    n += 1
    
# It is good practice to close a file when you are finished. 
infile.close()

# Calculate the mean.
mean = mean/n
print(mean)

### Conversion from text to numbers
<hr style="border: solid 1px red; margin-top: 1.5% ">

The lines read by Python are *text*, i.e. *strings*. To manipulate them (average them, plot them, *etc*.) we need to convert the strings to numbers, using either `float` or `int`. We saw in Lecture 5 that this conversion can fail, if the string does not look like what it should (an integer or a floating point number). This will notably happen if some data is missing, or if there are header lines instead of just numbers. However, you do not want your code to exit because of that: you will need to use a `try...except` construct to handle gracefully these errors and skip to the next line. 

## <span style="color:blue">Exercice 6.0:</span> Catch errors in conversions (Re: lecture 5)

The python built-in `float()` function tries to convert its input into a floating point number. This works if the input is already a float (`float(3.14)`, is an integer (`float(3)`), or even if it is a string containing a number `int("3.14")`. However, it will throw a `ValueError` if you try to convert a string that does not look like a number (try `float("qwe")`).

The goal of the exercise is to write a small function that will test if its argument can be converted to an int. Write a function called `isFloat`, that will return `True` if the argument can be converted, and `False` if it cannot. Use a `try...except` construct to catch exceptions raised by `float`.

In [ ]:
# your code here. Do not change variable names


In [ ]:
ok.grade('question-6_0')

### A more complex example
<hr style="border: solid 1px red; margin-top: 1.5% ">

Let's make this example more interesting. There is a **lot** of data out there for you to discover all kinds of interesting facts - you just need to be interested in learning a little analysis. For this case I have downloaded tidal gauge data for the port of Avonmouth from the [BODC](http://www.bodc.ac.uk/). Take some time now to open the file and have a look through it - [data/2012AVO.txt](data/2012AVO.txt) you will see the [metadata](http://en.wikipedia.org/wiki/Metadata):

```
Port:              P060
Site:              Avonmouth
Latitude:          51.51089
Longitude:         -2.71497
Start Date:        01JAN2012-00.00.00
End Date:          30APR2012-23.45.00
Contributor:       National Oceanography Centre, Liverpool
Datum information: The data refer to Admiralty Chart Datum (ACD)
Parameter code:    ASLVTD02 = Surface elevation (unspecified datum) of the water body by fixed in-situ pressure sensor```

Let's read the column ASLVTD02 (the surface elevation) and plot it. Pay attention to how we use `try...except` to detect lines that we want to skip.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

tide_file = open("data/2012AVO.txt", "r")

# Initialise an lists to store the elevation and the time
elevation = []
time = []
line_number = 0

# We read all the lines and first assume they are all data lines.
# If a line does not contain data in the right format (header, error...), 
# one of the operations that we are doing will fail, which we can catch
# with a try...except construct, and skip to the next line.

for line in tide_file:
    # Split this line into words. 
    try:
        words = line.split()
    except:
        # split will fail if the line is empty or contains just one word
        # in this case, skip to the next line
        continue # go directly to the next item in the loop, i.e. the next line

    # If we do not have 5 words then the line must be part of the header.
    if len(words)!=5:
        continue # go directly to the next line
    
    # The elevation data is on the 4th column. However, the BODC
    # appends a "M" when a value is improbable and an "N" when
    # data is missing (maybe a ship dumped into it during rough weather!)
    # We use a try/except block, to catch these cases when the conversion
    # to a float is not possible.
    try:
        level = float(words[3])
    except:
        continue
    elevation.append(level)

    # There is a measurement every quarter hour.
    time.append(line_number * 0.25/24)
    line_number += 1

    
# For plotting lets convert the list to a NumPy array.
elevation = np.array(elevation)
time = np.array(time)

fig = plt.figure()
plt.plot(time, elevation)
plt.xlabel("timestamp")
plt.ylabel("Elevation (meters)")
plt.show()

Quiz time:

* What tidal constituents can you identify by looking at this plot?
* Is this primarily a diurnal or semi-diurnal tidal region? (hint - change the x-axis range on the plot above).

### Split
<hr style="border: solid 1px red; margin-top: 1.5% ">

You will notice in the above example that we used the `split()` string member function. This is a very useful function for grabbing individual words on a line. When called without any arguments it assumes that the [delimiter](http://en.wikipedia.org/wiki/Delimiter) is a blank space. However, you can use this to split a string with any delimiter, *e.g.*, `line.split(';')`, `line.split(':')`.

### `Continue`
<hr style="border: solid 1px red; margin-top: 1.5% ">

We also used the `continue` statement. In a loop, (a `for` or `while` loop), it will be used to ignore the instructions placed below and skip to the next item in the loop:

In [ ]:
for i in range(5):
    print(i, ", above the continue")
    if i == 3:
        continue
    print(i, ", below the continue")

## <span style="color:blue">Exercice 6.1:</span>  Read a one-column data file

The file [data/file1.txt](data/file1.txt) contains a single column of floating point numbers. Make a program that reads the file and stores the numbers into a list. Convert this list into a numpy array called `column_float`. Modify your program to also convert the floating point numbers into integers and store them in an array called `column_int`.

In [ ]:
# Write your code here, do not change variable names for testing purposes

# import numpy as np

# ...

# column_float =
# column_int =

In [ ]:
ok.grade("question-6_1")

## <span style="color:blue">Exercice 6.2:</span>  Read a two-column data file

The file [data/xy.dat](data/xy.dat) contains two columns of numbers, corresponding to *x* and *y* coordinates on a curve. The start of the file looks like this:

-1.0000   -0.0000 <br>
-0.9933   -0.0087 <br>
-0.9867   -0.0179 <br>
-0.9800   -0.0274 <br>
-0.9733   -0.0374 <br>

Make a program that reads the first column into a list `xlist` and the second column into a list `ylist`. Then convert the lists to arrays named `xarray` and `yarray`, and plot the curve. Store the maximum and minimum y coordinates in two variables named `ymin` and `ymax`. (Hint: Read the file line by line, split each line into words, convert to float, and append to `xlist` and `ylist`.)</br>

In [ ]:
grade = ok.grade('question-6_2')

## <span style="color:blue">Exercice 6.3:</span>  Read a data file with headers and bad lines
The files [data/density_water.dat](https://raw.githubusercontent.com/ggorman/Introduction-to-programming-for-geoscientists/master/notebook/data/density_water.dat) and [data/density_air.dat](https://raw.githubusercontent.com/ggorman/Introduction-to-programming-for-geoscientists/master/notebook/data/density_air.dat) contain data about the density of water and air (respectively) for different temperatures. The data files have some comment lines starting with # and some lines are blank. The rest of the lines contain density data: the temperature in the first column and the corresponding density in the second column. The goal of this exercise is to read the data in such a file, discard commented or blank lines, and plot the density versus the temperature as distinct (small) circles for each data point. Write a function `readTempDenFile` that takes a filename as argument and returns two lists containing respectively the temperature and the density. Call this function on both files, and store the temperature and density in lists called `temp_air_list`, `dens_air_list`, `temp_water_list` and `dens_water_list`.

In [ ]:
grade = ok.grade("question-6_3")

## <span style="color:blue">Exercice 6.4:</span>  Read a file with missing data

The file [data/measurements.dat](data/measurements.dat)  is made of 3 columns corresponding to 2 different measurements taken at different times: *time* in hours, *meas_A* and *meas_B*. Sometimes, some of the measurements are missing, so there are fewer than 3 columns. Inspect the file visually and check its structure.

Write some code that reads the file, and stores the time and *meas_A* in two arrays called `hours_array` and `measA_array`. You should skip all the lines where data is missing.

Plot `measA_array` against `hours_array`.

In [ ]:
ok.grade("question-6_4")

## <span style="color:blue">Exercise 6.5 (**):</span> Read acceleration data and find velocities
A file [data/acc.dat](./data/acc.dat) contains measurements $a_0, a_1, \ldots, a_{n-1}$ of the acceleration of an object moving along a straight line. The measurement $a_k$ is taken at time point $t_k = k\Delta t$, where $\Delta t$ is the time spacing between the measurements. The purpose of the exercise is to load the acceleration data into a program and compute the velocity $v(t)$ of the object at some time $t$.

In general, the acceleration $a(t)$ is related to the velocity $v(t)$ through $v^\prime(t) = a(t)$. This means that

$$
v(t) = v(0) + \int_0^t{a(\tau)d\tau}
$$

If $a(t)$ is only known at some discrete, equally spaced points in time, $a_0, \ldots, a_{n-1}$ (which is the case in this exercise), we must compute the integral above numerically, for example by the Trapezoidal rule:

$$
v(t_k) \approx v(0) + \Delta t \left(\frac{1}{2}a_0 + \frac{1}{2}a_k + \sum_{i=1}^{k-1}a_i \right), \ \ 1 \leq k \leq n-1. 
$$

We assume $v(0) = 0$ so that also $v_0 = 0$.

1. Read the values $a_0, \ldots, a_{n-1}$ from file into an array `acc_array` and plot the acceleration versus time for $\Delta_t = 0.5$. The time should be stored in an array named `time_array`.

2. Then write a function `compute_velocity(dt, k, a)` that takes as arguments a time interval $\Delta_t$ `dt`, an index `k` and a list of accelerations `a`, uses the Trapezoidal rule to compute one $v(t_k)$ value and returns this value. Experiment with different values of $\Delta t$ and $k$.

In [ ]:
grade = ok.grade('question-6_5')

## File writing
<hr style="border: solid 2px red; margin-top: 1.5% ">

Writing a file in Python is simple. You just collect the text you want to write in one or more strings and, for each string, use a statement along the lines of

```python
outfile.write(string)```

The write function does not add a newline character so you may have to do that explicitly:

```python
outfile.write(string + ’\n’)```

That’s it! Compose the strings and write! Let's do an example. Write a nested list (table) to a file:

In [ ]:
# Let's define some table of data
data = [[ 0.75,        0.29619813, -0.29619813, -0.75      ],
        [ 0.29619813,  0.11697778, -0.11697778, -0.29619813],
        [-0.29619813, -0.11697778,  0.11697778,  0.29619813],
        [-0.75,       -0.29619813,  0.29619813,  0.75      ]]

# Open the file for writing. Notice the "w" indicates we are writing!
outfile = open("tmp_table.dat", "w")
for row in data:
    for column in row:
        outfile.write("%14.8f" % column)
    outfile.write("\n")   # ensure newline
outfile.close()

And that's it - run the above cell and take a look at the file that was generated in your Azure library clone.

## <span style="color:blue">Exercice 6.6:</span>  Write function data to a file

We want to dump $x$ and $f(x)$ values to a file named `function_data.dat`, where the $x$ values appear in the first column and the $f(x)$ values appear in the second. Choose $n$ equally spaced $x$ values in the interval [-4, 4]. Here, the function $f(x)$ is given by:

$f(x) = \frac{1}{\sqrt{2\pi}}\exp(-0.5x^2)$

In [ ]:
ok.grade('question-6_6')

In [ ]:
 ok.score()